<a href="https://colab.research.google.com/github/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting/blob/main/notebooks/xgboost_base_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from google.colab import userdata
token = userdata.get('GITHUB_TOKEN')
user_name = userdata.get('GITHUB_USERNAME')
mail = userdata.get('GITHUB_MAIL')

!git config --global user.name "{user_name}"
!git config --global user.email "{mail}"
!git clone https://{token}@github.com/azhgh22/Walmart-Recruiting-Store-Sales-Forecasting.git

%cd Walmart-Recruiting-Store-Sales-Forecasting

Cloning into 'Walmart-Recruiting-Store-Sales-Forecasting'...
remote: Enumerating objects: 196, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (126/126), done.
remote: Total 196 (delta 90), reused 159 (delta 64), pack-reused 0 (from 0)
Receiving objects: 100% (196/196), 6.38 MiB | 30.99 MiB/s, done.
Resolving deltas: 100% (90/90), done.
/content/Walmart-Recruiting-Store-Sales-Forecasting


In [3]:
from google.colab import userdata
!pip install -r requirements.txt
kaggle_json_path = userdata.get('KAGGLE_JSON_PATH')
! ./src/data_loader.sh -f {kaggle_json_path}

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.0/261.0 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 75.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 74.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from src.config import *

stores = pd.read_csv(STORES_PATH)
features = pd.read_csv(FEATURES_PATH)
train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

## **Merge tables and split data**

In [6]:
merged_train = pd.merge(train,stores,on='Store',how='left').merge(features,how='left',on=['Store','Date','IsHoliday'])

In [22]:
from src.time_series_split import TimeSeriesSplit
from src.config import SPLIT_DATE
merged_train.Date = pd.to_datetime(merged_train.Date)
x_train, x_val = TimeSeriesSplit(SPLIT_DATE).split(merged_train)
for i in range(1,3):
  x_train[f'shift{i}'] = x_train.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(i)
  x_val[f'shift{i}'] = x_val.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(i)

y_train = x_train.pop('Weekly_Sales')
y_val = x_val.pop('Weekly_Sales')

# **Time series features**

In [23]:
from feature_engineering.time_features import FeatureAdder

# deal with **NaN** values

In [24]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class NaImputer(BaseEstimator, TransformerMixin):
  def __init__(self, strategy='mean'):
    self.strategy = strategy
    self.imputer = SimpleImputer(strategy=strategy)
    self.na_cols = []

  def fit(self, X, y=None):
    self.na_cols = [col for col in X.columns if X[col].isna().sum() > 0]
    self.imputer.fit(X[self.na_cols])
    return self

  def transform(self, X, y=None):
    x_copy = X.copy()
    x_copy[self.na_cols] = self.imputer.transform(x_copy[self.na_cols])
    return x_copy

# **Cat2Num**

In [25]:
from sklearn.base import BaseEstimator, TransformerMixin

class Cat2Num(BaseEstimator, TransformerMixin):
  def fit(self, X, y=None):
    return self

  def transform(self, X, y=None):
    x_copy = X.copy()
    x_copy.pop('Date')
    x_copy.IsHoliday = x_copy.IsHoliday.astype(int)
    x_copy.Type = x_copy.Type.astype('category').cat.codes
    return x_copy

# **Add lags to data**

In [89]:
test = pd.read_csv(TEST_PATH)

In [105]:
test

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False
...,...,...,...,...
115059,45,98,2013-06-28,False
115060,45,98,2013-07-05,False
115061,45,98,2013-07-12,False
115062,45,98,2013-07-19,False


In [100]:
transformed_train.columns

Index(['Store', 'Dept', 'IsHoliday', 'Type', 'Size', 'Temperature',
       'Fuel_Price', 'MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4',
       'MarkDown5', 'CPI', 'Unemployment', 'DateDummy', 'shift1', 'shift2',
       'Month', 'Year', 'WeekOfYear', 'Is_Christmas', 'Is_SuperBowl',
       'Is_LaborDay', 'Is_Thanksgiving', 'Days_until_next_holiday',
       'Days_since_last_holiday', 'week_sin', 'week_cos', 'month_sin',
       'month_cos', 'Days_until_next_Christmas', 'Days_since_last_Christmas',
       'Days_until_next_SuperBowl', 'Days_since_last_SuperBowl',
       'Days_until_next_LaborDay', 'Days_since_last_LaborDay',
       'Days_until_next_Thanksgiving', 'Days_since_last_Thanksgiving'],
      dtype='object')

In [120]:
# sorted(transformed_train['DateDummy'].unique())
transformed_train.loc[transformed_train['DateDummy'] == i,'Store']

,Store
122587,13
200515,21
390590,41
62724,7
128511,14
...,...
23316,3
107712,11
207276,22
217393,23


In [158]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class GroupMeanImputer(BaseEstimator, TransformerMixin):
    def __init__(self, group_cols=['Store', 'Dept'], target_cols=None, fallback=0):
        self.group_cols = group_cols
        self.target_cols = target_cols
        self.fallback = fallback
        self.group_means_ = None

    def fit(self, X, y=None):
        X = X.copy()
        if self.target_cols is None:
            self.target_cols = X.select_dtypes(include='number').columns[
                X.isna().any()
            ].tolist()

        self.group_means_ = (
            X.groupby(self.group_cols)[self.target_cols]
            .mean()
            .reset_index()
        )
        return self

    def transform(self, X):
        X = X.copy()
        for col in self.target_cols:
            # Merge group means
            means = self.group_means_[[*self.group_cols, col]]
            X = X.merge(means, on=self.group_cols, how='left', suffixes=('', '_group_mean'))

            # Fill NaN with group mean, then fallback
            X[col] = X[col].fillna(X[f'{col}_group_mean'])
            X[col] = X[col].fillna(self.fallback)

            # Drop helper column
            X.drop(columns=[f'{col}_group_mean'], inplace=True)
        return X


In [180]:
from sklearn.base import BaseEstimator, TransformerMixin
import pandas as pd

class LagAdder(BaseEstimator, TransformerMixin):
  def __init__(self, val ,model ,lag_num:int = 2) -> None:
    super().__init__()
    self.lag_num = lag_num
    self.model = model
    self.y_val = val
    self.na_imputer = GroupMeanImputer()
    self.time = {}

  def fit(self, x:pd.DataFrame, y:pd.DataFrame):
    x_ = x.copy()
    y_ = pd.DataFrame(y.copy())
    y_['Store'] = x_['Store']
    y_['DateDummy'] = x_['DateDummy']
    y_['Dept'] = x_['Dept']
    for i in range(1,self.lag_num+1):
      x_[f'shift{i}'] = y_.groupby(['Store', 'Dept'])['Weekly_Sales'].shift(i)

    self.na_imputer.fit(x_)
    x_ = self.na_imputer.transform(x_)
    self.model.fit(x_,y)

    self.time = {}
    dates = sorted(x['DateDummy'].unique())
    for i in dates:
      cur_data = y_[y_['DateDummy'] == i][['Store', 'Dept','Weekly_Sales']].copy()
      self.time[i] = cur_data
    return self

  def transform(self, x:pd.DataFrame):
    answer = x.copy()
    answer['WeeklySales'] = 0
    x_ = x.copy()

    dates = sorted(x['DateDummy'].unique())

    time = self.time

    for i in dates:
      # print(i)
      cur_data = x_[x_['DateDummy'] == i].copy()
      for j in range(1,self.lag_num+1):
        if time.get(i-j) is not None:
          cur_data[f'shift{j}'] = cur_data.merge(time[i-j],how='left',on=['Store','Dept','DateDummy'])['Weekly_Sales']
        else:
          cur_data[f'shift{j}'] = np.nan

      # print(len(cur_data))
      cur_data = self.na_imputer.transform(cur_data)
      assert cur_data.isna().sum().sum() == 0
      # print(len(cur_data))
      pred = self.model.predict(cur_data)
      # print(len(answer.loc[answer['DateDummy'] == i,'WeeklySales']))
      # print(len(pred))
      assert len(answer.loc[answer['DateDummy'] == i,'WeeklySales'])==len(pred)
      answer.loc[answer['DateDummy'] == i,'WeeklySales'] = pred
      t_i = x_.loc[x_['DateDummy'] == i][['Store','Dept','DateDummy']]
      t_i['Weekly_Sales'] = pred
      time[i] = t_i

    return answer['WeeklySales']

  def predict(self, x:pd.DataFrame):
    return self.transform(x)

# **XGboost**

In [28]:
from xgboost import DMatrix
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_absolute_error
from src.utils import wmae

# **Grid search**

In [ ]:
from src.cross_validation import manual_model_search
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('adder', FeatureAdder()),
    ('imputer', NaImputer()),
    ('cat2num', Cat2Num()),
    # ('model', XGBRegressor(
    #     n_estimators=1000,
    #     learning_rate=0.1,
    #     max_depth=7,
    #     reg_lambda=3,
    #     min_split_loss=100,
    #     objective='reg:squarederror',
    #     random_state=42,
    # ))
  ])


mod = Pipeline([
    ('model', XGBRegressor(
        n_estimators=1000,
        learning_rate=0.1,
        max_depth=7,
        reg_lambda=3,
        min_split_loss=100,
        objective='reg:squarederror',
        random_state=42,
    ))
  ])


transformed_train = pipeline.fit_transform(x_train,y_train)
transformed_val = pipeline.transform(x_val)



param_grid = {
    'model__n_estimators': [200,500,800,1000],
    'model__learning_rate': [0.1],
    'model__max_depth': [7],
    'model__reg_lambda': [3],
    'model__min_split_loss': [100],
}


metric_kwargs = {
    'is_holiday': transformed_val['IsHoliday']
}

best_model, best_params, best_score = manual_model_search(
    model=mod,
    param_grid=param_grid,
    X_train=transformed_train,
    y_train=y_train,
    X_valid=transformed_val,
    y_valid=y_val,
    metric_func=wmae,
    metric_kwargs=metric_kwargs
)

print("\nBest Params:", best_params)
print("Best Validation Score:", best_score)

# **Model**

In [183]:
from sklearn.pipeline import Pipeline

xgb = XGBRegressor(
        n_estimators=1000,
        learning_rate=0.1,
        max_depth=7,
        # reg_lambda=1600,
        # min_split_loss=100,
        objective='reg:squarederror',
        random_state=42,
    )

pipeline = Pipeline([
    # ('adder', FeatureAdder(add_dummy_date=True,start_date=pd.Timestamp('2010-02-05'))),
    # ('imputer', NaImputer()),
    # ('cat2num', Cat2Num()),
    ('model', LagAdder(y_val,xgb,3))
  ])

model = pipeline.fit(transformed_train, y_train)


y_train_predict = model.predict(transformed_train)
y_val_predict = model.predict(transformed_val)

train_score = wmae(y_train, y_train_predict,x_train['IsHoliday'].to_list())
val_score = wmae(y_val, y_val_predict,x_val['IsHoliday'].to_list())
print(f"Train wmae: {train_score}, Val wmae: {val_score}")

/tmp/ipython-input-180-2797298325.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[24452.754   15552.883    3307.6436  ...  2632.7996    -39.44721
 14830.475  ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  answer.loc[answer['DateDummy'] == i,'WeeklySales'] = pred
/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(
/tmp/ipython-input-180-2797298325.py:59: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[  238.2762 63495.156  12973.666  ... 30741.438  24646.297  61688.03  ]' has dtype incompatible with int64, please explicitly cast to

Train wmae: 2393.46395790019, Val wmae: 3062.442065819097


/usr/local/lib/python3.11/dist-packages/sklearn/pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


# **upload to WanDB**

In [ ]:
! wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [ ]:
import wandb
import joblib

joblib.dump(model, "xgb_pipeline.pkl")
wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="xgboost:run1")

wandb.config.update({
    'merge1' : 'train, store, how=left, on=Store',
    'merge2' : 'train, features, how=left, on=Store, Date, IsHoliday',
    'merged_tables' : ['train','stores','features'],
    'time_features' : [
        'DateDummy', 'Month', 'Year',
        'WeekOfYear', 'Is_Christmas', 'Is_LaborDay', 'Is_Thanksgiving',
        'Is_SuperBowl', 'Days_until_next_holiday', 'Days_since_last_holiday',
        'week_sin', 'week_cos', 'month_sin', 'month_cos',
        'Days_until_next_Christmas', 'Days_since_last_Christmas',
        'Days_until_next_LaborDay', 'Days_since_last_LaborDay',
        'Days_until_next_Thanksgiving', 'Days_since_last_Thanksgiving',
        'Days_until_next_SuperBowl', 'Days_since_last_SuperBowl'
    ],
    'score_metric' : 'WMAE',
    'score_policy' : {
        'weight on holidays' : 5,
        'weight on non_holidays' : 1
    },
    'model' : 'Xgboost',
    'n_estimators' : 1000,
    'learning_rate' : 0.1,
    'max_depth' : 7,
    'reg_lambda' : 3,
    'min_split_loss' : 100,
    'objective' : 'reg:squarederror',
})

wandb.log({
    'train_wmae': train_score,
    'val_wmae': val_score
})


artifact = wandb.Artifact(
    name="xgb_pipeline",
    type="model",
    description="XGBoost pipeline with Date engineering and imputing"
)

artifact.add_file("xgb_pipeline.pkl")
wandb.log_artifact(artifact)

wandb.finish()

wandb: Currently logged in as: azhgh22 (MLBeasts) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


train_wmae,▁
val_wmae,▁
train_wmae,1578.84506
val_wmae,2848.68957


# **Load model from wandb**

In [ ]:
import wandb
import joblib

# Resume or init run
run = wandb.init(project="Walmart Recruiting - Store Sales Forecasting", name="xgboost:run1")

# Download the artifact
artifact = run.use_artifact('MLBeasts/Walmart Recruiting - Store Sales Forecasting/xgb_pipeline:latest', type='model')
artifact_dir = artifact.download()

# Load the model
model = joblib.load(f"{artifact_dir}/xgb_pipeline.pkl")

wandb:   1 of 1 files downloaded.  


In [ ]:
test = pd.read_csv(TEST_PATH)
merged_test = test.merge(stores,on='Store',how='left').merge(features,how='left',on=['Store','Date','IsHoliday'])
model.predict(merged_test)

array([33550.23    , 18070.236   , 17630.887   , ...,   887.0808  ,
         770.0637  ,   106.878555], dtype=float32)